In [ ]:
import pandas as pd
import numpy as np
import torch
import spacy
import re
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_pickle("complete_doxing_dataset.pkl")

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
train, test = train_test_split(df,stratify=df['label'], test_size=0.2)  # create stratified train and test data

In [ ]:
test['label'].value_counts()

In [ ]:
labels = train['label'].tolist()
tweets = train['tweet'].tolist()

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
inputs = tokenizer(tweets, padding="max_length", truncation=True)

In [ ]:
class twitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor (val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
    
    
train_dataset = twitterDataset(inputs,labels)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model.to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=20)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset)


In [ ]:
trainer.train()

In [ ]:
def save(model):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
    }, "output_model.pth")

save(model)

In [ ]:
def to_check_result(test_encoding) :
    input_ids = torch.tensor(test_encoding["input_ids"]).to(device)
    attention_mask = torch.tensor(test_encoding["attention_mask"]).to(device)
    with torch.no_grad() :
        outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
        y = np.argmax(outputs[0].to('cpu').numpy())
        
        return y

In [ ]:
test_preds = []

for i in test['tweet']:
    test_encoding = tokenizer(i, truncation=True, padding=True)
    input_ids = torch.tensor(test_encoding["input_ids"]).to(device)
    attention_mask = torch.tensor(test_encoding["attention_mask"]).to(device)
    op = to_check_result(test_encoding)    

    test_preds.append(op)

In [ ]:
test_preds.count(1)

In [ ]:
from sklearn.metrics import classification_report  ### test set
from sklearn.metrics import confusion_matrix

In [ ]:
target_names = ['benign', 'doxed']
print(classification_report(test['label'].tolist(), test_preds, target_names=target_names))

In [ ]:
tn, fp, fn, tp = confusion_matrix(test['label'].tolist(), test_preds).ravel()

In [ ]:
tpr = tp / (tp+fn)
tnr = tn / (tn+fp)
fpr = 1 - tnr
fnr = 1 - tpr
print(round(tpr, 2), round(tnr,2), round(fpr,2), round(fnr, 2))